# UMAP clustering for the methylation signature Position Weight Matrices
## Wastewater data

In [ ]:
import pandas as pd
import numpy as np
import warnings

from matplotlib.colors import ListedColormap

import os 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import umap
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
import seaborn as sns
import os
from PIL import Image, ImageFont
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans
import matplotlib.lines as mlines


seed = 98
import matplotlib.pyplot as plt
import seaborn as sns

# EFF1 preanalysis
## Import data
### > 50 lines in .gff

In [ ]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_WW_above50/'

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/EFF1_matrices_top50/flattened/EFF1_concat_matrices_top50.tsv'
EFF1_matrices = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [ ]:
print(EFF1_matrices.shape[0])

In [ ]:
print(EFF1_matrices.shape)
EFF1_matrices.head()

In [ ]:
EFF1_df = EFF1_matrices.loc[(EFF1_matrices.iloc[:, :492] != 0).any(axis=1)]

In [ ]:
EFF1_df['sample'].value_counts()
print(EFF1_df.iloc[:, :-1])

## Attach metadata
### Mod counts

In [ ]:
# Bring the mod counts from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/EFF1_contigs/EFF1_mod_counts.txt'

EFF1_df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(EFF1_df_mod_counts.shape[0])
EFF1_df_mod_counts.head()

In [ ]:
# Append to merged_data.tsv
EFF1_df_ext = EFF1_df.copy()
EFF1_df_ext.head()
EFF1_df_mod_counts.head()

# Reorder to match
EFF1_df_ordered = EFF1_df_mod_counts.loc[EFF1_df_ext.index]

# Check the min mod counts
EFF1_df_ordered['mod_count'].min()

In [ ]:
# Log transform
EFF1_df_ordered['mod_count_log'] = np.log2(EFF1_df_ordered['mod_count'])

EFF1_df_mod_counts = pd.concat([EFF1_df_ext, EFF1_df_ordered], axis=1)
print(EFF1_df_mod_counts.iloc[:, :-3])
EFF1_df_mod_counts.head()
EFF1_df = EFF1_df_mod_counts.copy()

### ARG counts (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/EFF1_contigs/EFF1_ARG_counts.txt'

EFF1_df_ARG_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(EFF1_df_ARG_counts.shape[0])
EFF1_df_ARG_counts.head()

In [ ]:
## Append to merged_data.tsv
EFF1_df_ext = EFF1_df.copy()
EFF1_df_ext.head()
EFF1_df_ARG_counts.head()

# Reorder to match
EFF1_df_ordered = EFF1_df_ARG_counts.loc[EFF1_df_ext.index]

EFF1_df_ARG_counts = pd.concat([EFF1_df_ext, EFF1_df_ordered], axis=1)
print(EFF1_df_ARG_counts.iloc[:, :-4])
EFF1_df_ARG_counts.head()
EFF1_df = EFF1_df_ARG_counts.copy()

### ARG names (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/EFF1_contigs/EFF1_ARG_names.txt'

EFF1_df_ARG_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(EFF1_df_ARG_names.shape[0])
EFF1_df_ARG_names.head()

# Replace NaN with empty
EFF1_df_ARG_names = EFF1_df_ARG_names.fillna('')

In [ ]:
## Append to merged_data.tsv
EFF1_df_ext = EFF1_df.copy()
EFF1_df_ext.head()
EFF1_df_ARG_names.head()

# Reorder to match
EFF1_df_ordered = EFF1_df_ARG_names.loc[EFF1_df_ext.index]

EFF1_df_ARG_names = pd.concat([EFF1_df_ext, EFF1_df_ordered], axis=1)
print(EFF1_df_ARG_names.iloc[:, :-5])
EFF1_df_ARG_names.head()
EFF1_df = EFF1_df_ARG_names.copy()

### Contig lengths

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/EFF1_contigs/EFF1_contigs_lengths.txt'

EFF1_df_contigs_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(EFF1_df_contigs_lengths.shape[0])
EFF1_df_contigs_lengths.head()

In [ ]:
## Append to merged_data.tsv
EFF1_df_ext = EFF1_df.copy()
EFF1_df_ext.head()
EFF1_df_contigs_lengths.head()

# Reorder to match
EFF1_df_ordered = EFF1_df_contigs_lengths.loc[EFF1_df_ext.index]

# Log transform
EFF1_df_ordered['length_sqrt'] = np.sqrt(EFF1_df_ordered['length'])

# Concat
EFF1_df_contigs_lengths = pd.concat([EFF1_df_ext, EFF1_df_ordered], axis=1)
print(EFF1_df_contigs_lengths.iloc[:, :-7])
EFF1_df_contigs_lengths.head()
EFF1_df = EFF1_df_contigs_lengths.copy()

### fARGene results

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/EFF1_contigs/EFF1_fARGene_names.txt'

EFF1_df_fARGene_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(EFF1_df_fARGene_names.shape[0])
EFF1_df_fARGene_names.head()

# Replace NaN with empty
EFF1_df_fARGene_names = EFF1_df_fARGene_names.fillna('')
EFF1_df_fARGene_names.head()

In [ ]:
## Append to merged_data.tsv
EFF1_df_ext = EFF1_df.copy()
EFF1_df_ext.head()
EFF1_df_fARGene_names.head()

# Reorder to match
EFF1_df_ordered = EFF1_df_fARGene_names.loc[EFF1_df_ext.index]

EFF1_df_fARGene_names = pd.concat([EFF1_df_ext, EFF1_df_ordered], axis=1)
print(EFF1_df_fARGene_names.iloc[:, :-8])
EFF1_df_fARGene_names.head()
EFF1_df = EFF1_df_fARGene_names.copy()

### Explore ARGs

In [ ]:
# Print those with erm(F)_3
erm_F = EFF1_df[EFF1_df['ARG_name'].str.contains('erm(F)_3', case=False, na=False, regex=False)]
print(erm_F)

In [ ]:
EFF1_df.head()
#print(EFF1_df.iloc[:, :-8])
#print(EFF1_df.index)

## Draw UMAP

In [ ]:
n_neighbors = [20]
min_dist = [0.1]
#colors = [0, 1, 2, 3]
#color_map = {0: '#8ce6e9', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
#custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(EFF1_df.iloc[:, :-8])
        EFF1_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': EFF1_df.index,
            'mod_count':EFF1_df['mod_count'],
            'mod_count_log':EFF1_df['mod_count_log'],
            'ARG_name':EFF1_df['ARG_name'],
            'ARG_count':EFF1_df['ARG_count'],
            'contig_length':EFF1_df['length'],
            'contig_length_sqrt':EFF1_df['length_sqrt'],
            'fARGene_class':EFF1_df['fARGene']
        })

        #EFF1_UMAP_df['ARG_count'] = EFF1_UMAP_df['ARG_count'].astype(str)
        
        fig = px.scatter(EFF1_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='mod_count_log',
                            #color='ARG_count',
                            title=f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            #color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        #fig.write_image(f'UMAP_WW_above50/EFF1_UMAP_{n}_{m}_mod_counts_lengths.png')
        #fig.write_html(f'UMAP_WW_above50/EFF1_UMAP_{n}_{m}_mod_counts_lengths.html')

## Exclude 'möykky'

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
EFF1_UMAP_df_focused = EFF1_UMAP_df.loc[(EFF1_UMAP_df['UMAP1']>= 3) & (EFF1_UMAP_df['UMAP1']<= 15)
    & (EFF1_UMAP_df['UMAP2']>= 1.5) & (EFF1_UMAP_df['UMAP2']<= 17)]

# Check
EFF1_UMAP_df_focused.head()

EFF1_df_focused = EFF1_df[EFF1_df.index.isin(EFF1_UMAP_df_focused['contig'])]
print(EFF1_df_focused)

In [ ]:
# Save contig IDs
EFF1_focused_contigs = EFF1_df_focused.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'EFF1_focused_contigs.txt')

with open(file_path, 'w') as file:
    for item in EFF1_focused_contigs:
        file.write(f"{item}\n")

In [ ]:
print(EFF1_df_focused.iloc[:, :-8])

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#b4f3f5', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(EFF1_df_focused.iloc[:, :-8])
        EFF1_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': EFF1_df_focused.index,
            'mod_count':EFF1_df_focused['mod_count'],
            'mod_count_log':EFF1_df_focused['mod_count_log'],
            'ARG_name':EFF1_df_focused['ARG_name'],
            'ARG_count':EFF1_df_focused['ARG_count'],
            'contig_length':EFF1_df_focused['length'],
            'contig_length_sqrt':EFF1_df_focused['length_sqrt'],
            'fARGene_class':EFF1_df_focused['fARGene']
        })

        EFF1_UMAP_df_focused['ARG_count'] = EFF1_UMAP_df_focused['ARG_count'].astype(str)

        fig = px.scatter(EFF1_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='ARG_count',
                            title=f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()

## Save data

In [ ]:
# Check
#EFF1_df_focused.head()
#EFF1_UMAP_df_focused.head()

In [ ]:
# Save data
EFF1_df_focused.to_csv('UMAP_WW_above50/EFF1_df_focused.csv', sep='\t', index=True)
EFF1_UMAP_df_focused.to_csv('UMAP_WW_above50/EFF1_UMAP_df_focused.csv', sep='\t', index=True)

# EFF1: Read in and plot

In [ ]:
# Read data
EFF1_df_focused = pd.read_csv('UMAP_WW_above50/EFF1_df_focused.csv', sep='\t', index_col=0, low_memory=False)
EFF1_UMAP_df_focused = pd.read_csv('UMAP_WW_above50/EFF1_UMAP_df_focused.csv', sep='\t', index_col=0, low_memory=False)

# Check
EFF1_df_focused.head()
EFF1_UMAP_df_focused.head()

## Plot ARG counts

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#b4f3f5', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

EFF1_UMAP_df_focused['ARG_count'] = EFF1_UMAP_df_focused['ARG_count'].astype(str)

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(EFF1_UMAP_df_focused.iloc[:, :-8])
        
fig = px.scatter(EFF1_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='ARG_count',
                 title=f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/EFF1_UMAP_{n}_{m}_focused_ARG_counts.png')
#fig.write_html(f'UMAP_WW_above50/EFF1_UMAP_{n}_{m}_focused_ARG_counts.html')

### Extract clusters
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
EFF1_df_focused_C1 = EFF1_UMAP_df_focused.loc[(EFF1_UMAP_df_focused['UMAP1']>= 2) & (EFF1_UMAP_df_focused['UMAP1']<= 2.2)
    & (EFF1_UMAP_df_focused['UMAP2']>= 3.6) & (EFF1_UMAP_df_focused['UMAP2']<= 3.7)]

# Check
EFF1_df_focused_C1.head()

EFF1_df_C1 = EFF1_df_focused[EFF1_df_focused.index.isin(EFF1_df_focused_C1['contig'])]
print(EFF1_df_C1)

In [ ]:
# Save contig IDs
EFF1_C1_contigs = EFF1_df_C1.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'EFF1_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in EFF1_C1_contigs:
        file.write(f"{item}\n")

## Plot fARGene

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = ['', 'class_a', 'class_b_1_2', 'class_c', 'class_d_1', 'class_d_2', 'mph', 'qnr', 'tet_efflux',
          'tet_rpg', 'tet_enzyme', 'erm_type_a', 'erm_type_f', 'aminoglycoside_model_a', 'aminoglycoside_model_b',
          'aminoglycoside_model_c', 'aminoglycoside_model_d', 'aminoglycoside_model_e', 'aminoglycoside_model_f',
          'aminoglycoside_model_g', 'aminoglycoside_model_h', 'aminoglycoside_model_i']

color_map = {'': '#8ce6e9', 'class_a': "#fabefa", 'class_b_1_2': "#d86950", 'class_c': "#ef360c", 'class_d_1': "#a27faf", 'class_d_2': "#c308a4", 'mph': "#e8db16",
             'qnr': "black", 'tet_efflux': "#04c60a", 'tet_rpg': "#1e7e21", 'tet_enzyme': "#779e78", 'erm_type_a': "#66e4e4", 'erm_type_f': "#25a5a5",
             'aminoglycoside_model_a': "#e8db16", 'aminoglycoside_model_b': "#1656e8", 'aminoglycoside_model_c': "#0f378e",
             'aminoglycoside_model_d': "#86a4eb", 'aminoglycoside_model_e': "#5b48d8", 'aminoglycoside_model_f': "#146eb4",
             'aminoglycoside_model_g': "#6f87f3", 'aminoglycoside_model_h': "#85baec", 'aminoglycoside_model_i': "#04bdfe"}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

EFF1_UMAP_df_focused['fARGene_class'] = EFF1_UMAP_df_focused['fARGene_class'].astype(str)
EFF1_UMAP_df_focused['fARGene_class'] = EFF1_UMAP_df_focused['fARGene_class'].replace('nan', '')

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(EFF1_UMAP_df_focused.iloc[:, :-8])

fig = px.scatter(EFF1_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='fARGene_class',
                 title=f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater EFF1 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/EFF1_UMAP_{n}_{m}_fARGene.png')
#fig.write_html(f'UMAP_WW_above50/EFF1_UMAP_{n}_{m}_fARGene.html')

### Extract clusters 
#### C1f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
EFF1_df_focused_C1f = EFF1_UMAP_df_focused.loc[(EFF1_UMAP_df_focused['UMAP1']>= 2.3) & (EFF1_UMAP_df_focused['UMAP1']<= 2.35)
    & (EFF1_UMAP_df_focused['UMAP2']>= 1.6) & (EFF1_UMAP_df_focused['UMAP2']<= 1.65)]

# Check
EFF1_df_focused_C1f.head()

EFF1_df_C1f = EFF1_df_focused[EFF1_df_focused.index.isin(EFF1_df_focused_C1f['contig'])]
print(EFF1_df_C1f)

In [ ]:
# Save contig IDs
EFF1_C1f_contigs = EFF1_df_C1f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'EFF1_C1f_contigs.txt')

with open(file_path, 'w') as file:
    for item in EFF1_C1f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C2f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
EFF1_df_focused_C2f = EFF1_UMAP_df_focused.loc[(EFF1_UMAP_df_focused['UMAP1']>= 2.8) & (EFF1_UMAP_df_focused['UMAP1']<= 3)
    & (EFF1_UMAP_df_focused['UMAP2']>= 4.2) & (EFF1_UMAP_df_focused['UMAP2']<= 4.3)]

# Check
EFF1_df_focused_C2f.head()

EFF1_df_C2f = EFF1_df_focused[EFF1_df_focused.index.isin(EFF1_df_focused_C2f['contig'])]
print(EFF1_df_C2f)

In [ ]:
# Save contig IDs
EFF1_C2f_contigs = EFF1_df_C2f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'EFF1_C2f_contigs.txt')

with open(file_path, 'w') as file:
    for item in EFF1_C2f_contigs:
        file.write(f"{item}\n")

# INF1 preanalysis
## Import data
### > 50 lines in .gff

In [ ]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_WW_above50/'

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF1_matrices_top50/flattened/INF1_concat_matrices_top50.tsv'
INF1_matrices = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [ ]:
print(INF1_matrices.shape[0])

In [ ]:
print(INF1_matrices.shape)
INF1_matrices.head()

In [ ]:
INF1_df = INF1_matrices.loc[(INF1_matrices.iloc[:, :492] != 0).any(axis=1)]

In [ ]:
INF1_df['sample'].value_counts()
print(INF1_df.iloc[:, :-1])

## Attach metadata
### Mod counts

In [ ]:
# Bring the mod counts from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF1_contigs/INF1_mod_counts.txt'

INF1_df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF1_df_mod_counts.shape[0])
INF1_df_mod_counts.head()

In [ ]:
# Append to merged_data.tsv
INF1_df_ext = INF1_df.copy()
INF1_df_ext.head()
INF1_df_mod_counts.head()

# Reorder to match
INF1_df_ordered = INF1_df_mod_counts.loc[INF1_df_ext.index]

# Check the min mod counts
INF1_df_ordered['mod_count'].min()

In [ ]:
# Log transform
INF1_df_ordered['mod_count_log'] = np.log2(INF1_df_ordered['mod_count'])

INF1_df_mod_counts = pd.concat([INF1_df_ext, INF1_df_ordered], axis=1)
print(INF1_df_mod_counts.iloc[:, :-3])
INF1_df_mod_counts.head()
INF1_df = INF1_df_mod_counts.copy()

### ARG counts (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF1_contigs/INF1_ARG_counts.txt'

INF1_df_ARG_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF1_df_ARG_counts.shape[0])
INF1_df_ARG_counts.head()

In [ ]:
## Append to merged_data.tsv
INF1_df_ext = INF1_df.copy()
INF1_df_ext.head()
INF1_df_ARG_counts.head()

# Reorder to match
INF1_df_ordered = INF1_df_ARG_counts.loc[INF1_df_ext.index]

INF1_df_ARG_counts = pd.concat([INF1_df_ext, INF1_df_ordered], axis=1)
print(INF1_df_ARG_counts.iloc[:, :-4])
INF1_df_ARG_counts.head()
INF1_df = INF1_df_ARG_counts.copy()

### ARG names (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF1_contigs/INF1_ARG_names.txt'

INF1_df_ARG_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF1_df_ARG_names.shape[0])
INF1_df_ARG_names.head()

# Replace NaN with empty
INF1_df_ARG_names = INF1_df_ARG_names.fillna('')

In [ ]:
## Append to merged_data.tsv
INF1_df_ext = INF1_df.copy()
INF1_df_ext.head()
INF1_df_ARG_names.head()

# Reorder to match
INF1_df_ordered = INF1_df_ARG_names.loc[INF1_df_ext.index]

INF1_df_ARG_names = pd.concat([INF1_df_ext, INF1_df_ordered], axis=1)
print(INF1_df_ARG_names.iloc[:, :-5])
INF1_df_ARG_names.head()
INF1_df = INF1_df_ARG_names.copy()

### Contig lengths

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF1_contigs/INF1_contigs_lengths.txt'

INF1_df_contigs_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF1_df_contigs_lengths.shape[0])
INF1_df_contigs_lengths.head()

In [ ]:
## Append to merged_data.tsv
INF1_df_ext = INF1_df.copy()
INF1_df_ext.head()
INF1_df_contigs_lengths.head()

# Reorder to match
INF1_df_ordered = INF1_df_contigs_lengths.loc[INF1_df_ext.index]

# Log transform
INF1_df_ordered['length_sqrt'] = np.sqrt(INF1_df_ordered['length'])

# Concat
INF1_df_contigs_lengths = pd.concat([INF1_df_ext, INF1_df_ordered], axis=1)
print(INF1_df_contigs_lengths.iloc[:, :-7])
INF1_df_contigs_lengths.head()
INF1_df = INF1_df_contigs_lengths.copy()

### fARGene results

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF1_contigs/INF1_fARGene_names.txt'

INF1_df_fARGene_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF1_df_fARGene_names.shape[0])
INF1_df_fARGene_names.head()

# Replace NaN with empty
INF1_df_fARGene_names = INF1_df_fARGene_names.fillna('')
INF1_df_fARGene_names.head()

In [ ]:
## Append to merged_data.tsv
INF1_df_ext = INF1_df.copy()
INF1_df_ext.head()
INF1_df_fARGene_names.head()

# Reorder to match
INF1_df_ordered = INF1_df_fARGene_names.loc[INF1_df_ext.index]

INF1_df_fARGene_names = pd.concat([INF1_df_ext, INF1_df_ordered], axis=1)
print(INF1_df_fARGene_names.iloc[:, :-8])
INF1_df_fARGene_names.head()
INF1_df = INF1_df_fARGene_names.copy()

### Explore ARGs

In [ ]:
# Print those with erm(F)_3
erm_F = INF1_df[INF1_df['ARG_name'].str.contains('erm(F)_3', case=False, na=False, regex=False)]
print(erm_F)

In [ ]:
INF1_df.head()
#print(INF1_df.iloc[:, :-8])
#print(INF1_df.index)

## Draw UMAP

In [ ]:
n_neighbors = [20]
min_dist = [0.1]
#colors = [0, 1, 2, 3]
#color_map = {0: '#8ce6e9', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
#custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF1_df.iloc[:, :-8])
        INF1_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': INF1_df.index,
            'mod_count':INF1_df['mod_count'],
            'mod_count_log':INF1_df['mod_count_log'],
            'ARG_name':INF1_df['ARG_name'],
            'ARG_count':INF1_df['ARG_count'],
            'contig_length':INF1_df['length'],
            'contig_length_sqrt':INF1_df['length_sqrt'],
            'fARGene_class':INF1_df['fARGene']
        })

        #INF1_UMAP_df['ARG_count'] = INF1_UMAP_df['ARG_count'].astype(str)
        
        fig = px.scatter(INF1_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='mod_count_log',
                            #color='ARG_count',
                            title=f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            #color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        fig.write_image(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_mod_counts_lengths.png')
        fig.write_html(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_mod_counts_lengths.html')

## Exclude 'möykky'

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_UMAP_df_focused = INF1_UMAP_df.loc[(INF1_UMAP_df['UMAP1']>= 1) & (INF1_UMAP_df['UMAP1']<= 15)
    & (INF1_UMAP_df['UMAP2']>= -4) & (INF1_UMAP_df['UMAP2']<= 15)]

# Check
INF1_UMAP_df_focused.head()

INF1_df_focused = INF1_df[INF1_df.index.isin(INF1_UMAP_df_focused['contig'])]
print(INF1_df_focused)

In [ ]:
# Save contig IDs
INF1_focused_contigs = INF1_df_focused.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_focused_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_focused_contigs:
        file.write(f"{item}\n")

In [ ]:
print(INF1_df_focused.iloc[:, :-8])

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#dcd377', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF1_df_focused.iloc[:, :-8])
        INF1_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': INF1_df_focused.index,
            'mod_count':INF1_df_focused['mod_count'],
            'mod_count_log':INF1_df_focused['mod_count_log'],
            'ARG_name':INF1_df_focused['ARG_name'],
            'ARG_count':INF1_df_focused['ARG_count'],
            'contig_length':INF1_df_focused['length'],
            'contig_length_sqrt':INF1_df_focused['length_sqrt'],
            'fARGene_class':INF1_df_focused['fARGene']
        })

        INF1_UMAP_df_focused['ARG_count'] = INF1_UMAP_df_focused['ARG_count'].astype(str)

        fig = px.scatter(INF1_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='ARG_count',
                            title=f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()

## Save data

In [ ]:
# Check
INF1_df_focused.head()
INF1_UMAP_df_focused.head()

In [ ]:
# Save data
INF1_df_focused.to_csv('UMAP_WW_above50/INF1_df_focused.csv', sep='\t', index=True)
INF1_UMAP_df_focused.to_csv('UMAP_WW_above50/INF1_UMAP_df_focused.csv', sep='\t', index=True)

# INF1: Read in and plot

In [ ]:
# Read data
INF1_df_focused = pd.read_csv('UMAP_WW_above50/INF1_df_focused.csv', sep='\t', index_col=0, low_memory=False)
INF1_UMAP_df_focused = pd.read_csv('UMAP_WW_above50/INF1_UMAP_df_focused.csv', sep='\t', index_col=0, low_memory=False)

# Check
INF1_df_focused.head()
INF1_UMAP_df_focused.head()

## Plot ARG counts

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#dcd377', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

INF1_UMAP_df_focused['ARG_count'] = INF1_UMAP_df_focused['ARG_count'].astype(str)
category_order = ["0", "1", "2", "3"]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF1_UMAP_df_focused.iloc[:, :-8])
        
fig = px.scatter(INF1_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='ARG_count',
                 title=f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}',
                 category_orders={'ARG_count': category_order},
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_focused_ARG_counts.png')
#fig.write_html(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_focused_ARG_counts.html')

### Extract clusters
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C1 = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= 5) & (INF1_UMAP_df_focused['UMAP1']<= 10)
    & (INF1_UMAP_df_focused['UMAP2']>= 5) & (INF1_UMAP_df_focused['UMAP2']<= 10)]

# Check
INF1_df_focused_C1.head()

INF1_df_C1 = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C1['contig'])]
print(INF1_df_C1)

In [ ]:
# Save contig IDs
INF1_C1_contigs = INF1_df_C1.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C1_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C2

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C2 = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= 6.9) & (INF1_UMAP_df_focused['UMAP1']<= 7.2)
    & (INF1_UMAP_df_focused['UMAP2']>= 14.5) & (INF1_UMAP_df_focused['UMAP2']<= 14.6)]

# Check
INF1_df_focused_C2.head()

INF1_df_C2 = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C2['contig'])]
print(INF1_df_C2)

In [ ]:
# Save contig IDs
INF1_C2_contigs = INF1_df_C2.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C2_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C2_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C3

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C3 = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= 2.6) & (INF1_UMAP_df_focused['UMAP1']<= 2.8)
    & (INF1_UMAP_df_focused['UMAP2']>= 2.1) & (INF1_UMAP_df_focused['UMAP2']<= 2.2)]

# Check
INF1_df_focused_C3.head()

INF1_df_C3 = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C3['contig'])]
print(INF1_df_C3)

In [ ]:
# Save contig IDs
INF1_C3_contigs = INF1_df_C3.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C3_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C3_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C4

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C4 = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= -1.69) & (INF1_UMAP_df_focused['UMAP1']<= -1.635)
    & (INF1_UMAP_df_focused['UMAP2']>= -0.99) & (INF1_UMAP_df_focused['UMAP2']<= -0.93)]

# Check
INF1_df_focused_C4.head()

INF1_df_C4 = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C4['contig'])]
print(INF1_df_C4)

In [ ]:
# Save contig IDs
INF1_C4_contigs = INF1_df_C4.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C4_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C4_contigs:
        file.write(f"{item}\n")

## Plot fARGene

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = ['', 'class_a', 'class_b_1_2', 'class_c', 'class_d_1', 'class_d_2', 'mph', 'qnr', 'tet_efflux',
          'tet_rpg', 'tet_enzyme', 'erm_type_a', 'erm_type_f', 'aminoglycoside_model_a', 'aminoglycoside_model_b',
          'aminoglycoside_model_c', 'aminoglycoside_model_d', 'aminoglycoside_model_e', 'aminoglycoside_model_f',
          'aminoglycoside_model_g', 'aminoglycoside_model_h', 'aminoglycoside_model_i']

color_map = {'': '#dcd377', 'class_a': "#fabefa", 'class_b_1_2': "#d86950", 'class_c': "#ef360c", 'class_d_1': "#a27faf", 'class_d_2': "#c308a4", 'mph': "#e8db16",
             'qnr': "black", 'tet_efflux': "#04c60a", 'tet_rpg': "#1e7e21", 'tet_enzyme': "#779e78", 'erm_type_a': "#66e4e4", 'erm_type_f': "#25a5a5",
             'aminoglycoside_model_a': "#e8db16", 'aminoglycoside_model_b': "#1656e8", 'aminoglycoside_model_c': "#0f378e",
             'aminoglycoside_model_d': "#86a4eb", 'aminoglycoside_model_e': "#5b48d8", 'aminoglycoside_model_f': "#146eb4",
             'aminoglycoside_model_g': "#6f87f3", 'aminoglycoside_model_h': "#85baec", 'aminoglycoside_model_i': "#04bdfe"}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

INF1_UMAP_df_focused['fARGene_class'] = INF1_UMAP_df_focused['fARGene_class'].astype(str)
INF1_UMAP_df_focused['fARGene_class'] = INF1_UMAP_df_focused['fARGene_class'].replace('nan', '')

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF1_UMAP_df_focused.iloc[:, :-8])

fig = px.scatter(INF1_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='fARGene_class',
                 title=f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_fARGene.png')
#fig.write_html(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_fARGene.html')

### Extract clusters 
#### C1f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C1f = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= 3.65) & (INF1_UMAP_df_focused['UMAP1']<= 3.75)
    & (INF1_UMAP_df_focused['UMAP2']>= 4.53) & (INF1_UMAP_df_focused['UMAP2']<= 4.62)]

# Check
INF1_df_focused_C1f.head()

INF1_df_C1f = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C1f['contig'])]
print(INF1_df_C1f)

In [ ]:
# Save contig IDs
INF1_C1f_contigs = INF1_df_C1f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C1f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C1f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C2f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C2f = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= -2.045) & (INF1_UMAP_df_focused['UMAP1']<= -1.9)
    & (INF1_UMAP_df_focused['UMAP2']>= 7.51) & (INF1_UMAP_df_focused['UMAP2']<= 7.6)]

# Check
INF1_df_focused_C2f.head()

INF1_df_C2f = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C2f['contig'])]
print(INF1_df_C2f)

In [ ]:
# Save contig IDs
INF1_C2f_contigs = INF1_df_C2f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C2f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C2f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C3f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C3f = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= -2.35) & (INF1_UMAP_df_focused['UMAP1']<= -2.2)
    & (INF1_UMAP_df_focused['UMAP2']>= 7.65) & (INF1_UMAP_df_focused['UMAP2']<= 7.76)]

# Check
INF1_df_focused_C3f.head()

INF1_df_C3f = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C3f['contig'])]
print(INF1_df_C3f)

In [ ]:
# Save contig IDs
INF1_C3f_contigs = INF1_df_C3f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C3f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C3f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C4f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C4f = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= -0.6) & (INF1_UMAP_df_focused['UMAP1']<= -0.55)
    & (INF1_UMAP_df_focused['UMAP2']>= 9.96) & (INF1_UMAP_df_focused['UMAP2']<= 9.98)]

# Check
INF1_df_focused_C4f.head()

INF1_df_C4f = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C4f['contig'])]
print(INF1_df_C4f)

In [ ]:
# Save contig IDs
INF1_C4f_contigs = INF1_df_C4f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C4f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C4f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C5f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C5f = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= -2.8) & (INF1_UMAP_df_focused['UMAP1']<= -2.6)
    & (INF1_UMAP_df_focused['UMAP2']>= 9.88) & (INF1_UMAP_df_focused['UMAP2']<= 10)]

# Check
INF1_df_focused_C5f.head()

INF1_df_C5f = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C5f['contig'])]
print(INF1_df_C5f)

In [ ]:
# Save contig IDs
INF1_C5f_contigs = INF1_df_C5f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C5f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C5f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C6f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF1_df_focused_C6f = INF1_UMAP_df_focused.loc[(INF1_UMAP_df_focused['UMAP1']>= -2.6) & (INF1_UMAP_df_focused['UMAP1']<= -2.4)
    & (INF1_UMAP_df_focused['UMAP2']>= 7.2) & (INF1_UMAP_df_focused['UMAP2']<= 7.4)]

# Check
INF1_df_focused_C6f.head()

INF1_df_C6f = INF1_df_focused[INF1_df_focused.index.isin(INF1_df_focused_C6f['contig'])]
print(INF1_df_C6f)

In [ ]:
# Save contig IDs
INF1_C6f_contigs = INF1_df_C6f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF1_C6f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF1_C6f_contigs:
        file.write(f"{item}\n")

# INF2 preanalysis
## Import data
### > 50 lines in .gff

In [ ]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_WW_above50/'

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF2_matrices_top50/flattened/INF2_concat_matrices_top50.tsv'
INF2_matrices = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [ ]:
print(INF2_matrices.shape[0])

In [ ]:
print(INF2_matrices.shape)
INF2_matrices.head()

In [ ]:
INF2_df = INF2_matrices.loc[(INF2_matrices.iloc[:, :492] != 0).any(axis=1)]

In [ ]:
INF2_df['sample'].value_counts()
print(INF2_df.iloc[:, :-1])

## Attach metadata
### Mod counts

In [ ]:
# Bring the mod counts from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF2_contigs/INF2_mod_counts.txt'

INF2_df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF2_df_mod_counts.shape[0])
INF2_df_mod_counts.head()

In [ ]:
# Append to merged_data.tsv
INF2_df_ext = INF2_df.copy()
INF2_df_ext.head()
INF2_df_mod_counts.head()

# Reorder to match
INF2_df_ordered = INF2_df_mod_counts.loc[INF2_df_ext.index]

# Check the min mod counts
INF2_df_ordered['mod_count'].min()

In [ ]:
# Log transform
INF2_df_ordered['mod_count_log'] = np.log2(INF2_df_ordered['mod_count'])

INF2_df_mod_counts = pd.concat([INF2_df_ext, INF2_df_ordered], axis=1)
print(INF2_df_mod_counts.iloc[:, :-3])
INF2_df_mod_counts.head()
INF2_df = INF2_df_mod_counts.copy()

### ARG counts (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF2_contigs/INF2_ARG_counts.txt'

INF2_df_ARG_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF2_df_ARG_counts.shape[0])
INF2_df_ARG_counts.head()

In [ ]:
## Append to merged_data.tsv
INF2_df_ext = INF2_df.copy()
INF2_df_ext.head()
INF2_df_ARG_counts.head()

# Reorder to match
INF2_df_ordered = INF2_df_ARG_counts.loc[INF2_df_ext.index]

INF2_df_ARG_counts = pd.concat([INF2_df_ext, INF2_df_ordered], axis=1)
print(INF2_df_ARG_counts.iloc[:, :-4])
INF2_df_ARG_counts.head()
INF2_df = INF2_df_ARG_counts.copy()

### ARG names (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF2_contigs/INF2_ARG_names.txt'

INF2_df_ARG_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF2_df_ARG_names.shape[0])
INF2_df_ARG_names.head()

# Replace NaN with empty
INF2_df_ARG_names = INF2_df_ARG_names.fillna('')

In [ ]:
## Append to merged_data.tsv
INF2_df_ext = INF2_df.copy()
INF2_df_ext.head()
INF2_df_ARG_names.head()

# Reorder to match
INF2_df_ordered = INF2_df_ARG_names.loc[INF2_df_ext.index]

INF2_df_ARG_names = pd.concat([INF2_df_ext, INF2_df_ordered], axis=1)
print(INF2_df_ARG_names.iloc[:, :-5])
INF2_df_ARG_names.head()
INF2_df = INF2_df_ARG_names.copy()

### Contig lengths

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF2_contigs/INF2_contigs_lengths.txt'

INF2_df_contigs_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF2_df_contigs_lengths.shape[0])
INF2_df_contigs_lengths.head()

In [ ]:
## Append to merged_data.tsv
INF2_df_ext = INF2_df.copy()
INF2_df_ext.head()
INF2_df_contigs_lengths.head()

# Reorder to match
INF2_df_ordered = INF2_df_contigs_lengths.loc[INF2_df_ext.index]

# Log transform
INF2_df_ordered['length_sqrt'] = np.sqrt(INF2_df_ordered['length'])

# Concat
INF2_df_contigs_lengths = pd.concat([INF2_df_ext, INF2_df_ordered], axis=1)
print(INF2_df_contigs_lengths.iloc[:, :-7])
INF2_df_contigs_lengths.head()
INF2_df = INF2_df_contigs_lengths.copy()

### fARGene results

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF2_contigs/INF2_fARGene_names.txt'

INF2_df_fARGene_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF2_df_fARGene_names.shape[0])
INF2_df_fARGene_names.head()

# Replace NaN with empty
INF2_df_fARGene_names = INF2_df_fARGene_names.fillna('')
INF2_df_fARGene_names.head()

In [ ]:
## Append to merged_data.tsv
INF2_df_ext = INF2_df.copy()
INF2_df_ext.head()
INF2_df_fARGene_names.head()

# Reorder to match
INF2_df_ordered = INF2_df_fARGene_names.loc[INF2_df_ext.index]

INF2_df_fARGene_names = pd.concat([INF2_df_ext, INF2_df_ordered], axis=1)
print(INF2_df_fARGene_names.iloc[:, :-8])
INF2_df_fARGene_names.head()
INF2_df = INF2_df_fARGene_names.copy()

### Explore ARGs

In [ ]:
# Print those with erm(F)_3
erm_F = INF2_df[INF2_df['ARG_name'].str.contains('erm(F)_3', case=False, na=False, regex=False)]
print(erm_F)

In [ ]:
INF2_df.head()
#print(INF1_df.iloc[:, :-8])
#print(INF1_df.index)

## Draw UMAP

In [ ]:
n_neighbors = [20]
min_dist = [0.1]
#colors = [0, 1, 2, 3]
#color_map = {0: '#8ce6e9', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
#custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF2_df.iloc[:, :-8])
        INF2_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': INF2_df.index,
            'mod_count':INF2_df['mod_count'],
            'mod_count_log':INF2_df['mod_count_log'],
            'ARG_name':INF2_df['ARG_name'],
            'ARG_count':INF2_df['ARG_count'],
            'contig_length':INF2_df['length'],
            'contig_length_sqrt':INF2_df['length_sqrt'],
            'fARGene_class':INF2_df['fARGene']
        })

        #INF2_UMAP_df['ARG_count'] = INF2_UMAP_df['ARG_count'].astype(str)
        
        fig = px.scatter(INF2_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='mod_count_log',
                            #color='ARG_count',
                            title=f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            #color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        fig.write_image(f'UMAP_WW_above50/INF2_UMAP_{n}_{m}_mod_counts_lengths.png')
        fig.write_html(f'UMAP_WW_above50/INF2_UMAP_{n}_{m}_mod_counts_lengths.html')

## Exclude 'möykky'

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF2_UMAP_df_focused = INF2_UMAP_df.loc[(INF2_UMAP_df['UMAP1']>= 1) & (INF2_UMAP_df['UMAP1']<= 15)
    & (INF2_UMAP_df['UMAP2']>= -4) & (INF2_UMAP_df['UMAP2']<= 15)]

# Check
INF2_UMAP_df_focused.head()

INF2_df_focused = INF2_df[INF2_df.index.isin(INF2_UMAP_df_focused['contig'])]
print(INF2_df_focused)

In [ ]:
# Save contig IDs
INF2_focused_contigs = INF2_df_focused.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF2_focused_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF2_focused_contigs:
        file.write(f"{item}\n")

In [ ]:
print(INF2_df_focused.iloc[:, :-8])

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#dcd377', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF2_df_focused.iloc[:, :-8])
        INF2_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': INF2_df_focused.index,
            'mod_count':INF2_df_focused['mod_count'],
            'mod_count_log':INF2_df_focused['mod_count_log'],
            'ARG_name':INF2_df_focused['ARG_name'],
            'ARG_count':INF2_df_focused['ARG_count'],
            'contig_length':INF2_df_focused['length'],
            'contig_length_sqrt':INF2_df_focused['length_sqrt'],
            'fARGene_class':INF2_df_focused['fARGene']
        })

        INF2_UMAP_df_focused['ARG_count'] = INF2_UMAP_df_focused['ARG_count'].astype(str)

        fig = px.scatter(INF2_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='ARG_count',
                            title=f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()

## Save data

In [ ]:
# Check
INF2_df_focused.head()
INF2_UMAP_df_focused.head()

In [ ]:
# Save data
INF2_df_focused.to_csv('UMAP_WW_above50/INF2_df_focused.csv', sep='\t', index=True)
INF2_UMAP_df_focused.to_csv('UMAP_WW_above50/INF2_UMAP_df_focused.csv', sep='\t', index=True)

# INF2: Read in and plot

In [ ]:
# Read data
INF2_df_focused = pd.read_csv('UMAP_WW_above50/INF2_df_focused.csv', sep='\t', index_col=0, low_memory=False)
INF2_UMAP_df_focused = pd.read_csv('UMAP_WW_above50/INF2_UMAP_df_focused.csv', sep='\t', index_col=0, low_memory=False)

# Check
INF2_df_focused.head()
INF2_UMAP_df_focused.head()

## Plot ARG counts

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#dcd377', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

INF2_UMAP_df_focused['ARG_count'] = INF2_UMAP_df_focused['ARG_count'].astype(str)
category_order = ["0", "1", "2", "3"]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF2_UMAP_df_focused.iloc[:, :-8])
        
fig = px.scatter(INF2_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='ARG_count',
                 title=f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}',
                 category_orders={'ARG_count': category_order},
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/INF2_UMAP_{n}_{m}_focused_ARG_counts.png')
#fig.write_html(f'UMAP_WW_above50/INF2_UMAP_{n}_{m}_focused_ARG_counts.html')

### Extract clusters
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF2_df_focused_C1 = INF2_UMAP_df_focused.loc[(INF2_UMAP_df_focused['UMAP1']>= 5) & (INF2_UMAP_df_focused['UMAP1']<= 10)
    & (INF2_UMAP_df_focused['UMAP2']>= 5) & (INF2_UMAP_df_focused['UMAP2']<= 10)]

# Check
INF2_df_focused_C1.head()

INF2_df_C1 = INF2_df_focused[INF2_df_focused.index.isin(INF2_df_focused_C1['contig'])]
print(INF2_df_C1)

In [ ]:
# Save contig IDs
INF2_C1_contigs = INF2_df_C1.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF2_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF2_C1_contigs:
        file.write(f"{item}\n")

## Plot fARGene

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = ['', 'class_a', 'class_b_1_2', 'class_c', 'class_d_1', 'class_d_2', 'mph', 'qnr', 'tet_efflux',
          'tet_rpg', 'tet_enzyme', 'erm_type_a', 'erm_type_f', 'aminoglycoside_model_a', 'aminoglycoside_model_b',
          'aminoglycoside_model_c', 'aminoglycoside_model_d', 'aminoglycoside_model_e', 'aminoglycoside_model_f',
          'aminoglycoside_model_g', 'aminoglycoside_model_h', 'aminoglycoside_model_i']

color_map = {'': '#dcd377', 'class_a': "#fabefa", 'class_b_1_2': "#d86950", 'class_c': "#ef360c", 'class_d_1': "#a27faf", 'class_d_2': "#c308a4", 'mph': "#e8db16",
             'qnr': "black", 'tet_efflux': "#04c60a", 'tet_rpg': "#1e7e21", 'tet_enzyme': "#779e78", 'erm_type_a': "#66e4e4", 'erm_type_f': "#25a5a5",
             'aminoglycoside_model_a': "#e8db16", 'aminoglycoside_model_b': "#1656e8", 'aminoglycoside_model_c': "#0f378e",
             'aminoglycoside_model_d': "#86a4eb", 'aminoglycoside_model_e': "#5b48d8", 'aminoglycoside_model_f': "#146eb4",
             'aminoglycoside_model_g': "#6f87f3", 'aminoglycoside_model_h': "#85baec", 'aminoglycoside_model_i': "#04bdfe"}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

INF1_UMAP_df_focused['fARGene_class'] = INF1_UMAP_df_focused['fARGene_class'].astype(str)
INF1_UMAP_df_focused['fARGene_class'] = INF1_UMAP_df_focused['fARGene_class'].replace('nan', '')

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF1_UMAP_df_focused.iloc[:, :-8])

fig = px.scatter(INF1_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='fARGene_class',
                 title=f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}', 
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_fARGene.png')
#fig.write_html(f'UMAP_WW_above50/INF1_UMAP_{n}_{m}_fARGene.html')

### Extract clusters 
#### C1f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF2_df_focused_C1f = INF2_UMAP_df_focused.loc[(INF2_UMAP_df_focused['UMAP1']>= 3.65) & (INF2_UMAP_df_focused['UMAP1']<= 3.75)
    & (INF2_UMAP_df_focused['UMAP2']>= 4.53) & (INF2_UMAP_df_focused['UMAP2']<= 4.62)]

# Check
INF2_df_focused_C1f.head()

INF2_df_C1f = INF2_df_focused[INF2_df_focused.index.isin(INF2_df_focused_C1f['contig'])]
print(INF2_df_C1f)

In [ ]:
# Save contig IDs
INF2_C1f_contigs = INF2_df_C1f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF2_C1f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF2_C1f_contigs:
        file.write(f"{item}\n")

# INF3 preanalysis
## Import data
### > 50 lines in .gff

In [45]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_WW_above50/'

In [46]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF3_matrices_top50/flattened/INF3_concat_matrices_top50.tsv'
INF3_matrices = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [47]:
print(INF3_matrices.shape[0])

62794


In [48]:
print(INF3_matrices.shape)
INF3_matrices.head()

(62794, 493)


,-20_A_m4C,-19_A_m4C,-18_A_m4C,-17_A_m4C,-16_A_m4C,-15_A_m4C,-14_A_m4C,-13_A_m4C,-12_A_m4C,-11_A_m4C,...,12_G_m6A,13_G_m6A,14_G_m6A,15_G_m6A,16_G_m6A,17_G_m6A,18_G_m6A,19_G_m6A,20_G_m6A,sample
0,,,,,,,,,,,,,,,,,,,,,
s25827.ctg028425l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,INF3
s41612.ctg045875l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,INF3
s12315.ctg013652l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,INF3
s2863.ctg003227l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,INF3
s45324.ctg050095l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,INF3


In [49]:
INF3_df = INF3_matrices.loc[(INF3_matrices.iloc[:, :492] != 0).any(axis=1)]

In [50]:
INF3_df['sample'].value_counts()
print(INF3_df.iloc[:, :-1])

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s12315.ctg013652l   0.000000   0.000000   0.000000   0.000000   0.000000   
s45324.ctg050095l   0.000000   0.000000   0.000000   0.000000   0.000000   
s32964.ctg036264l  -0.063972  -0.055099  -0.249511   0.037640   0.221037   
s23535.ctg025924l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg042813l      0.000000   0.000000   0.000000   0.000000   0.000000   
...                      ...        ...        ...        ...        ...   
s10.ctg017524l      0.000000   0.000000   0.000000   0.000000   0.000000   
s35433.ctg038969l  -0.001173   0.351594  -0.074281   0.399949   0.066953   
s22394.ctg024659l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg048453l      0.000000   0.000000   0.000000   0.000000   0.000000   
s14440.ctg015990l   0.000000   0.000000   0.000000   0.000000   0.000000   

           

## Attach metadata
### Mod counts

In [51]:
# Bring the mod counts from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF3_contigs/INF3_mod_counts.txt'

INF3_df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF3_df_mod_counts.shape[0])
INF3_df_mod_counts.head()

62795


,mod_count
contig,
s54701.ctg061971l,23
s30279.ctg033331l,0
s18948.ctg020930l,73
s4159.ctg004690l,4
s3953.ctg004458l,157


In [52]:
# Append to merged_data.tsv
INF3_df_ext = INF3_df.copy()
INF3_df_ext.head()
INF3_df_mod_counts.head()

# Reorder to match
INF3_df_ordered = INF3_df_mod_counts.loc[INF3_df_ext.index]

# Check the min mod counts
INF3_df_ordered['mod_count'].min()

50

In [53]:
# Log transform
INF3_df_ordered['mod_count_log'] = np.log2(INF3_df_ordered['mod_count'])

INF3_df_mod_counts = pd.concat([INF3_df_ext, INF3_df_ordered], axis=1)
print(INF3_df_mod_counts.iloc[:, :-3])
INF3_df_mod_counts.head()
INF3_df = INF3_df_mod_counts.copy()

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s12315.ctg013652l   0.000000   0.000000   0.000000   0.000000   0.000000   
s45324.ctg050095l   0.000000   0.000000   0.000000   0.000000   0.000000   
s32964.ctg036264l  -0.063972  -0.055099  -0.249511   0.037640   0.221037   
s23535.ctg025924l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg042813l      0.000000   0.000000   0.000000   0.000000   0.000000   
...                      ...        ...        ...        ...        ...   
s10.ctg017524l      0.000000   0.000000   0.000000   0.000000   0.000000   
s35433.ctg038969l  -0.001173   0.351594  -0.074281   0.399949   0.066953   
s22394.ctg024659l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg048453l      0.000000   0.000000   0.000000   0.000000   0.000000   
s14440.ctg015990l   0.000000   0.000000   0.000000   0.000000   0.000000   

           

### ARG counts (ResFinder)

In [54]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF3_contigs/INF3_ARG_counts.txt'

INF3_df_ARG_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF3_df_ARG_counts.shape[0])
INF3_df_ARG_counts.head()

62795


,ARG_count
contig,
s54701.ctg061971l,0
s30279.ctg033331l,0
s18948.ctg020930l,0
s4159.ctg004690l,0
s3953.ctg004458l,0


In [55]:
## Append to merged_data.tsv
INF3_df_ext = INF3_df.copy()
INF3_df_ext.head()
INF3_df_ARG_counts.head()

# Reorder to match
INF3_df_ordered = INF3_df_ARG_counts.loc[INF3_df_ext.index]

INF3_df_ARG_counts = pd.concat([INF3_df_ext, INF3_df_ordered], axis=1)
print(INF3_df_ARG_counts.iloc[:, :-4])
INF3_df_ARG_counts.head()
INF3_df = INF3_df_ARG_counts.copy()

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s12315.ctg013652l   0.000000   0.000000   0.000000   0.000000   0.000000   
s45324.ctg050095l   0.000000   0.000000   0.000000   0.000000   0.000000   
s32964.ctg036264l  -0.063972  -0.055099  -0.249511   0.037640   0.221037   
s23535.ctg025924l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg042813l      0.000000   0.000000   0.000000   0.000000   0.000000   
...                      ...        ...        ...        ...        ...   
s10.ctg017524l      0.000000   0.000000   0.000000   0.000000   0.000000   
s35433.ctg038969l  -0.001173   0.351594  -0.074281   0.399949   0.066953   
s22394.ctg024659l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg048453l      0.000000   0.000000   0.000000   0.000000   0.000000   
s14440.ctg015990l   0.000000   0.000000   0.000000   0.000000   0.000000   

           

### ARG names (ResFinder)

In [56]:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF3_contigs/INF3_ARG_names.txt'

INF3_df_ARG_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF3_df_ARG_names.shape[0])
INF3_df_ARG_names.head()

# Replace NaN with empty
INF3_df_ARG_names = INF3_df_ARG_names.fillna('')

62797


In [57]:
## Append to merged_data.tsv
INF3_df_ext = INF3_df.copy()
INF3_df_ext.head()
INF3_df_ARG_names.head()

# Reorder to match
INF3_df_ordered = INF3_df_ARG_names.loc[INF3_df_ext.index]

INF3_df_ARG_names = pd.concat([INF3_df_ext, INF3_df_ordered], axis=1)
print(INF3_df_ARG_names.iloc[:, :-5])
INF3_df_ARG_names.head()
INF3_df = INF3_df_ARG_names.copy()

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s12315.ctg013652l   0.000000   0.000000   0.000000   0.000000   0.000000   
s45324.ctg050095l   0.000000   0.000000   0.000000   0.000000   0.000000   
s32964.ctg036264l  -0.063972  -0.055099  -0.249511   0.037640   0.221037   
s23535.ctg025924l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg042813l      0.000000   0.000000   0.000000   0.000000   0.000000   
...                      ...        ...        ...        ...        ...   
s10.ctg017524l      0.000000   0.000000   0.000000   0.000000   0.000000   
s35433.ctg038969l  -0.001173   0.351594  -0.074281   0.399949   0.066953   
s22394.ctg024659l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg048453l      0.000000   0.000000   0.000000   0.000000   0.000000   
s14440.ctg015990l   0.000000   0.000000   0.000000   0.000000   0.000000   

           

### Contig lengths

In [58]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF3_contigs/INF3_contigs_lengths.txt'

INF3_df_contigs_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF3_df_contigs_lengths.shape[0])
INF3_df_contigs_lengths.head()

62921


,length
#name,
s0.ctg000001l,106757
s1.ctg000002l,36120
s2.ctg000003l,53502
s3.ctg000004l,15026
s4.ctg000005l,24959


In [59]:
## Append to merged_data.tsv
INF3_df_ext = INF3_df.copy()
INF3_df_ext.head()
INF3_df_contigs_lengths.head()

# Reorder to match
INF3_df_ordered = INF3_df_contigs_lengths.loc[INF3_df_ext.index]

# Log transform
INF3_df_ordered['length_sqrt'] = np.sqrt(INF3_df_ordered['length'])

# Concat
INF3_df_contigs_lengths = pd.concat([INF3_df_ext, INF3_df_ordered], axis=1)
print(INF3_df_contigs_lengths.iloc[:, :-7])
INF3_df_contigs_lengths.head()
INF3_df = INF3_df_contigs_lengths.copy()

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s12315.ctg013652l   0.000000   0.000000   0.000000   0.000000   0.000000   
s45324.ctg050095l   0.000000   0.000000   0.000000   0.000000   0.000000   
s32964.ctg036264l  -0.063972  -0.055099  -0.249511   0.037640   0.221037   
s23535.ctg025924l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg042813l      0.000000   0.000000   0.000000   0.000000   0.000000   
...                      ...        ...        ...        ...        ...   
s10.ctg017524l      0.000000   0.000000   0.000000   0.000000   0.000000   
s35433.ctg038969l  -0.001173   0.351594  -0.074281   0.399949   0.066953   
s22394.ctg024659l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg048453l      0.000000   0.000000   0.000000   0.000000   0.000000   
s14440.ctg015990l   0.000000   0.000000   0.000000   0.000000   0.000000   

           

### fARGene results

In [60]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/INF3_contigs/INF3_fARGene_names.txt'

INF3_df_fARGene_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(INF3_df_fARGene_names.shape[0])
INF3_df_fARGene_names.head()

# Replace NaN with empty
INF3_df_fARGene_names = INF3_df_fARGene_names.fillna('')
INF3_df_fARGene_names.head()

62795


,fARGene
contig,
s10145.ctg011266l,aminoglycoside_i
s10354.ctg011495l,mph
s10721.ctg011906l,beta_lactamase_b_1_2
s10.ctg000026l,beta_lactamase_d_2
s10.ctg000111l,beta_lactamase_d_2


In [61]:
## Append to merged_data.tsv
INF3_df_ext = INF3_df.copy()
INF3_df_ext.head()
INF3_df_fARGene_names.head()

# Reorder to match
INF3_df_ordered = INF3_df_fARGene_names.loc[INF3_df_ext.index]

INF3_df_fARGene_names = pd.concat([INF3_df_ext, INF3_df_ordered], axis=1)
print(INF3_df_fARGene_names.iloc[:, :-8])
INF3_df_fARGene_names.head()
INF3_df = INF3_df_fARGene_names.copy()

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s12315.ctg013652l   0.000000   0.000000   0.000000   0.000000   0.000000   
s45324.ctg050095l   0.000000   0.000000   0.000000   0.000000   0.000000   
s32964.ctg036264l  -0.063972  -0.055099  -0.249511   0.037640   0.221037   
s23535.ctg025924l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg042813l      0.000000   0.000000   0.000000   0.000000   0.000000   
...                      ...        ...        ...        ...        ...   
s10.ctg017524l      0.000000   0.000000   0.000000   0.000000   0.000000   
s35433.ctg038969l  -0.001173   0.351594  -0.074281   0.399949   0.066953   
s22394.ctg024659l   0.000000   0.000000   0.000000   0.000000   0.000000   
s10.ctg048453l      0.000000   0.000000   0.000000   0.000000   0.000000   
s14440.ctg015990l   0.000000   0.000000   0.000000   0.000000   0.000000   

           

### Explore ARGs

In [62]:
# Print those with erm(F)_3
erm_F = INF3_df[INF3_df['ARG_name'].str.contains('erm(F)_3', case=False, na=False, regex=False)]
print(erm_F)

                   -20_A_m4C  -19_A_m4C  -18_A_m4C  -17_A_m4C  -16_A_m4C  \
0                                                                          
s10.ctg005298l      0.019183   0.005249   0.032924   0.124167    0.04648   
s28016.ctg030850l   0.000000   0.000000   0.000000   0.000000    0.00000   

                   -15_A_m4C  -14_A_m4C  -13_A_m4C  -12_A_m4C  -11_A_m4C  ...  \
0                                                                         ...   
s10.ctg005298l     -0.082692  -0.067487  -0.129743  -0.008881  -0.008881  ...   
s28016.ctg030850l   0.000000   0.000000   0.000000   0.000000   0.000000  ...   

                   19_G_m6A  20_G_m6A  sample  mod_count  mod_count_log  \
0                                                                         
s10.ctg005298l    -0.278585  0.120568    INF3       2902      11.502832   
s28016.ctg030850l  0.000000  0.000000    INF3        423       8.724514   

                   ARG_count           ARG_name  length  length_sqrt 

In [63]:
INF3_df.head()
#print(INF1_df.iloc[:, :-8])
#print(INF1_df.index)

,-20_A_m4C,-19_A_m4C,-18_A_m4C,-17_A_m4C,-16_A_m4C,-15_A_m4C,-14_A_m4C,-13_A_m4C,-12_A_m4C,-11_A_m4C,...,19_G_m6A,20_G_m6A,sample,mod_count,mod_count_log,ARG_count,ARG_name,length,length_sqrt,fARGene
0,,,,,,,,,,,,,,,,,,,,,
s12315.ctg013652l,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,...,0.000000,0.000000,INF3,165,7.366322,0,,46274,215.113923,
s45324.ctg050095l,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,...,0.000000,0.000000,INF3,264,8.044394,0,,25234,158.852133,
s32964.ctg036264l,-0.063972,-0.055099,-0.249511,0.03764,0.221037,-0.147588,-0.037585,0.14445,-0.046304,0.15166,...,0.047794,-0.268181,INF3,4025,11.974773,0,,42539,206.249848,
s23535.ctg025924l,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,...,0.000000,0.000000,INF3,158,7.303781,0,,21151,145.433834,
s10.ctg042813l,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,...,0.000000,0.000000,INF3,155,7.276124,0,,29679,172.275941,


## Draw UMAP

In [ ]:
n_neighbors = [20]
min_dist = [0.1]
#colors = [0, 1, 2, 3]
#color_map = {0: '#8ce6e9', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
#custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF3_df.iloc[:, :-8])
        INF3_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': INF3_df.index,
            'mod_count':INF3_df['mod_count'],
            'mod_count_log':INF3_df['mod_count_log'],
            'ARG_name':INF3_df['ARG_name'],
            'ARG_count':INF3_df['ARG_count'],
            'contig_length':INF3_df['length'],
            'contig_length_sqrt':INF3_df['length_sqrt'],
            'fARGene_class':INF3_df['fARGene']
        })

        #INF3_UMAP_df['ARG_count'] = INF3_UMAP_df['ARG_count'].astype(str)
        
        fig = px.scatter(INF3_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='mod_count_log',
                            #color='ARG_count',
                            title=f' Wastewater INF2 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            #color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater INF3 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        fig.write_image(f'UMAP_WW_above50/INF3_UMAP_{n}_{m}_mod_counts_lengths.png')
        fig.write_html(f'UMAP_WW_above50/INF3_UMAP_{n}_{m}_mod_counts_lengths.html')

## Exclude 'möykky'

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF3_UMAP_df_focused = INF2_UMAP_df.loc[(INF3_UMAP_df['UMAP1']>= 1) & (INF3_UMAP_df['UMAP1']<= 15)
    & (INF3_UMAP_df['UMAP2']>= -4) & (INF3_UMAP_df['UMAP2']<= 15)]

# Check
INF3_UMAP_df_focused.head()

INF3_df_focused = INF3_df[INF3_df.index.isin(INF3_UMAP_df_focused['contig'])]
print(INF3_df_focused)

In [ ]:
# Save contig IDs
INF3_focused_contigs = INF3_df_focused.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF3_focused_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF3_focused_contigs:
        file.write(f"{item}\n")

In [ ]:
print(INF3_df_focused.iloc[:, :-8])

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#dcd377', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF3_df_focused.iloc[:, :-8])
        INF3_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': INF3_df_focused.index,
            'mod_count':INF3_df_focused['mod_count'],
            'mod_count_log':INF3_df_focused['mod_count_log'],
            'ARG_name':INF3_df_focused['ARG_name'],
            'ARG_count':INF3_df_focused['ARG_count'],
            'contig_length':INF3_df_focused['length'],
            'contig_length_sqrt':INF3_df_focused['length_sqrt'],
            'fARGene_class':INF3_df_focused['fARGene']
        })

        INF3_UMAP_df_focused['ARG_count'] = INF3_UMAP_df_focused['ARG_count'].astype(str)

        fig = px.scatter(INF3_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='ARG_count',
                            title=f' Wastewater INF3 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater INF3 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()

## Save data

In [ ]:
# Check
INF3_df_focused.head()
INF3_UMAP_df_focused.head()

In [ ]:
# Save data
INF3_df_focused.to_csv('UMAP_WW_above50/INF3_df_focused.csv', sep='\t', index=True)
INF3_UMAP_df_focused.to_csv('UMAP_WW_above50/INF3_UMAP_df_focused.csv', sep='\t', index=True)

# INF2: Read in and plot

In [ ]:
# Read data
INF3_df_focused = pd.read_csv('UMAP_WW_above50/INF3_df_focused.csv', sep='\t', index_col=0, low_memory=False)
INF3_UMAP_df_focused = pd.read_csv('UMAP_WW_above50/INF3_UMAP_df_focused.csv', sep='\t', index_col=0, low_memory=False)

# Check
INF3_df_focused.head()
INF3_UMAP_df_focused.head()

## Plot ARG counts

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#dcd377', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

INF3_UMAP_df_focused['ARG_count'] = INF3_UMAP_df_focused['ARG_count'].astype(str)
category_order = ["0", "1", "2", "3"]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF3_UMAP_df_focused.iloc[:, :-8])
        
fig = px.scatter(INF3_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='ARG_count',
                 title=f' Wastewater INF3 - UMAP with n_neighbors={n}, min_dist={m}',
                 category_orders={'ARG_count': category_order},
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater INF3 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/INF3_UMAP_{n}_{m}_focused_ARG_counts.png')
#fig.write_html(f'UMAP_WW_above50/INF3_UMAP_{n}_{m}_focused_ARG_counts.html')

### Extract clusters
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF3_df_focused_C1 = INF3_UMAP_df_focused.loc[(INF3_UMAP_df_focused['UMAP1']>= 5) & (INF3_UMAP_df_focused['UMAP1']<= 10)
    & (INF3_UMAP_df_focused['UMAP2']>= 5) & (INF3_UMAP_df_focused['UMAP2']<= 10)]

# Check
INF3_df_focused_C1.head()

INF3_df_C1 = INF3_df_focused[INF3_df_focused.index.isin(INF3_df_focused_C1['contig'])]
print(INF3_df_C1)

In [ ]:
# Save contig IDs
INF3_C1_contigs = INF3_df_C1.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF3_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF3_C1_contigs:
        file.write(f"{item}\n")

## Plot fARGene

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = ['', 'class_a', 'class_b_1_2', 'class_c', 'class_d_1', 'class_d_2', 'mph', 'qnr', 'tet_efflux',
          'tet_rpg', 'tet_enzyme', 'erm_type_a', 'erm_type_f', 'aminoglycoside_model_a', 'aminoglycoside_model_b',
          'aminoglycoside_model_c', 'aminoglycoside_model_d', 'aminoglycoside_model_e', 'aminoglycoside_model_f',
          'aminoglycoside_model_g', 'aminoglycoside_model_h', 'aminoglycoside_model_i']

color_map = {'': '#dcd377', 'class_a': "#fabefa", 'class_b_1_2': "#d86950", 'class_c': "#ef360c", 'class_d_1': "#a27faf", 'class_d_2': "#c308a4", 'mph': "#e8db16",
             'qnr': "black", 'tet_efflux': "#04c60a", 'tet_rpg': "#1e7e21", 'tet_enzyme': "#779e78", 'erm_type_a': "#66e4e4", 'erm_type_f': "#25a5a5",
             'aminoglycoside_model_a': "#e8db16", 'aminoglycoside_model_b': "#1656e8", 'aminoglycoside_model_c': "#0f378e",
             'aminoglycoside_model_d': "#86a4eb", 'aminoglycoside_model_e': "#5b48d8", 'aminoglycoside_model_f': "#146eb4",
             'aminoglycoside_model_g': "#6f87f3", 'aminoglycoside_model_h': "#85baec", 'aminoglycoside_model_i': "#04bdfe"}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

INF3_UMAP_df_focused['fARGene_class'] = INF3_UMAP_df_focused['fARGene_class'].astype(str)
INF3_UMAP_df_focused['fARGene_class'] = INF3_UMAP_df_focused['fARGene_class'].replace('nan', '')

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(INF1_UMAP_df_focused.iloc[:, :-8])

fig = px.scatter(INF1_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='fARGene_class',
                 title=f' Wastewater INF3 - UMAP with n_neighbors={n}, min_dist={m}', 
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater INF1 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/INF3_UMAP_{n}_{m}_fARGene.png')
#fig.write_html(f'UMAP_WW_above50/INF3_UMAP_{n}_{m}_fARGene.html')

### Extract clusters 
#### C1f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
INF3_df_focused_C1f = INF3_UMAP_df_focused.loc[(INF3_UMAP_df_focused['UMAP1']>= 3.65) & (INF3_UMAP_df_focused['UMAP1']<= 3.75)
    & (INF3_UMAP_df_focused['UMAP2']>= 4.53) & (INF3_UMAP_df_focused['UMAP2']<= 4.62)]

# Check
INF3_df_focused_C1f.head()

INF3_df_C1f = INF3_df_focused[INF3_df_focused.index.isin(INF3_df_focused_C1f['contig'])]
print(INF3_df_C1f)

In [ ]:
# Save contig IDs
INF3_C1f_contigs = INF3_df_C1f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'INF3_C1f_contigs.txt')

with open(file_path, 'w') as file:
    for item in INF3_C1f_contigs:
        file.write(f"{item}\n")

# SLU1 preanalysis
## Import data
### > 50 lines in .gff

In [ ]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_WW_above50/'

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU1_matrices_top50/flattened/SLU1_concat_matrices_top50.tsv'
SLU1_matrices = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [ ]:
print(SLU1_matrices.shape[0])

In [ ]:
print(SLU1_matrices.shape)
SLU1_matrices.head()

In [ ]:
SLU1_df = SLU1_matrices.loc[(SLU1_matrices.iloc[:, :492] != 0).any(axis=1)]
print(SLU1_df.shape)

In [ ]:
SLU1_df['sample'].value_counts()
print(SLU1_df.iloc[:, :-1])

## Attach metadata
### Mod counts

In [ ]:
# Bring the mod counts from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU1_contigs/SLU1_mod_counts.txt'

SLU1_df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU1_df_mod_counts.shape[0])
SLU1_df_mod_counts.head()

In [ ]:
# Append to merged_data.tsv
SLU1_df_ext = SLU1_df.copy()
SLU1_df_ext.head()
SLU1_df_mod_counts.head()

# Reorder to match
SLU1_df_ordered = SLU1_df_mod_counts.loc[SLU1_df_ext.index]

# Check the min mod counts
SLU1_df_ordered['mod_count'].min()

In [ ]:
# Log transform
SLU1_df_ordered['mod_count_log'] = np.log2(SLU1_df_ordered['mod_count'])

SLU1_df_mod_counts = pd.concat([SLU1_df_ext, SLU1_df_ordered], axis=1)
print(SLU1_df_mod_counts.iloc[:, :-3])
SLU1_df_mod_counts.head()
SLU1_df = SLU1_df_mod_counts.copy()

### ARG counts (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU1_contigs/SLU1_ARG_counts.txt'

SLU1_df_ARG_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU1_df_ARG_counts.shape[0])
SLU1_df_ARG_counts.head()

In [ ]:
## Append to merged_data.tsv
SLU1_df_ext = SLU1_df.copy()
SLU1_df_ext.head()
SLU1_df_ARG_counts.head()

# Reorder to match
SLU1_df_ordered = SLU1_df_ARG_counts.loc[SLU1_df_ext.index]

SLU1_df_ARG_counts = pd.concat([SLU1_df_ext, SLU1_df_ordered], axis=1)
print(SLU1_df_ARG_counts.iloc[:, :-4])
SLU1_df_ARG_counts.head()
SLU1_df = SLU1_df_ARG_counts.copy()

### ARG names (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU1_contigs/SLU1_ARG_names.txt'

SLU1_df_ARG_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU1_df_ARG_names.shape[0])
SLU1_df_ARG_names.head()

# Replace NaN with empty
SLU1_df_ARG_names = SLU1_df_ARG_names.fillna('')

In [ ]:
## Append to merged_data.tsv
SLU1_df_ext = SLU1_df.copy()
SLU1_df_ext.head()
SLU1_df_ARG_names.head()

# Reorder to match
SLU1_df_ordered = SLU1_df_ARG_names.loc[SLU1_df_ext.index]

SLU1_df_ARG_names = pd.concat([SLU1_df_ext, SLU1_df_ordered], axis=1)
print(SLU1_df_ARG_names.iloc[:, :-5])
SLU1_df_ARG_names.head()
SLU1_df = SLU1_df_ARG_names.copy()

### Contig lengths

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU1_contigs/SLU1_contigs_lengths.txt'

SLU1_df_contigs_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU1_df_contigs_lengths.shape[0])
SLU1_df_contigs_lengths.head()

In [ ]:
## Append to merged_data.tsv
SLU1_df_ext = SLU1_df.copy()
SLU1_df_ext.head()
SLU1_df_contigs_lengths.head()

# Reorder to match
SLU1_df_ordered = SLU1_df_contigs_lengths.loc[SLU1_df_ext.index]

# Log transform
SLU1_df_ordered['length_sqrt'] = np.sqrt(SLU1_df_ordered['length'])

# Concat
SLU1_df_contigs_lengths = pd.concat([SLU1_df_ext, SLU1_df_ordered], axis=1)
print(SLU1_df_contigs_lengths.iloc[:, :-7])
SLU1_df_contigs_lengths.head()
SLU1_df = SLU1_df_contigs_lengths.copy()

### fARGene results

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU1_contigs/SLU1_fARGene_names.txt'

SLU1_df_fARGene_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU1_df_fARGene_names.shape[0])
SLU1_df_fARGene_names.head()

# Replace NaN with empty
SLU1_df_fARGene_names = SLU1_df_fARGene_names.fillna('')
SLU1_df_fARGene_names.head()

In [ ]:
## Append to merged_data.tsv
SLU1_df_ext = SLU1_df.copy()
SLU1_df_ext.head()
SLU1_df_fARGene_names.head()

# Reorder to match
SLU1_df_ordered = SLU1_df_fARGene_names.loc[SLU1_df_ext.index]

SLU1_df_fARGene_names = pd.concat([SLU1_df_ext, SLU1_df_ordered], axis=1)
print(SLU1_df_fARGene_names.iloc[:, :-8])
SLU1_df_fARGene_names.head()
SLU1_df = SLU1_df_fARGene_names.copy()

### Explore ARGs

In [ ]:
# Print those with erm(F)_3
erm_F = SLU1_df[SLU1_df['ARG_name'].str.contains('erm(F)_3', case=False, na=False, regex=False)]
print(erm_F)

In [ ]:
SLU1_df.head()
#print(SLU1_df.iloc[:, :-8])
#print(SLU1_df.index)

## Draw UMAP

In [ ]:
n_neighbors = [20]
min_dist = [0.1]
#colors = [0, 1, 2, 3]
#color_map = {0: '#8ce6e9', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
#custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(SLU1_df.iloc[:, :-8])
        SLU1_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': SLU1_df.index,
            'mod_count':SLU1_df['mod_count'],
            'mod_count_log':SLU1_df['mod_count_log'],
            'ARG_name':SLU1_df['ARG_name'],
            'ARG_count':SLU1_df['ARG_count'],
            'contig_length':SLU1_df['length'],
            'contig_length_sqrt':SLU1_df['length_sqrt'],
            'fARGene_class':SLU1_df['fARGene']
        })

        SLU1_UMAP_df['ARG_count'] = SLU1_UMAP_df['ARG_count'].astype(str)
        
        fig = px.scatter(SLU1_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='mod_count_log',
                            #color='ARG_count',
                            title=f' Wastewater SLU1 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            #color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater SLU1 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        #fig.write_image(f'UMAP_WW_above50/SLU1_UMAP_{n}_{m}_mod_counts_lengths.png')
        #fig.write_html(f'UMAP_WW_above50/SLU1_UMAP_{n}_{m}_mod_counts_lengths.html')

## Exclude 'möykky'

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU1_UMAP_df_focused = SLU1_UMAP_df.loc[(SLU1_UMAP_df['UMAP1']>= 3) & (SLU1_UMAP_df['UMAP1']<= 15)
    & (SLU1_UMAP_df['UMAP2']>= 1.5) & (SLU1_UMAP_df['UMAP2']<= 17)]

# Check
SLU1_UMAP_df_focused.head()

SLU1_df_focused = SLU1_df[SLU1_df.index.isin(SLU1_UMAP_df_focused['contig'])]
print(SLU1_df_focused)

In [ ]:
# Save contig IDs
SLU1_focused_contigs = SLU1_df_focused.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU1_focused_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU1_focused_contigs:
        file.write(f"{item}\n")

In [ ]:
print(SLU1_df_focused.iloc[:, :-8])

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#b4f3f5', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(SLU1_df_focused.iloc[:, :-8])
        SLU1_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': SLU1_df_focused.index,
            'mod_count':SLU1_df_focused['mod_count'],
            'mod_count_log':SLU1_df_focused['mod_count_log'],
            'ARG_name':SLU1_df_focused['ARG_name'],
            'ARG_count':SLU1_df_focused['ARG_count'],
            'contig_length':SLU1_df_focused['length'],
            'contig_length_sqrt':SLU1_df_focused['length_sqrt'],
            'fARGene_class':SLU1_df_focused['fARGene']
        })

        SLU1_UMAP_df_focused['ARG_count'] = SLU1_UMAP_df_focused['ARG_count'].astype(str)

        fig = px.scatter(SLU1_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='ARG_count',
                            title=f' Wastewater SLU1 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater SLU1 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()

## Save data

In [ ]:
# Check
#SLU1_df_focused.head()
#SLU1_UMAP_df_focused.head()

In [ ]:
# Save data
SLU1_df_focused.to_csv('UMAP_WW_above50/SLU1_df_focused.csv', sep='\t', index=True)
SLU1_UMAP_df_focused.to_csv('UMAP_WW_above50/SLU1_UMAP_df_focused.csv', sep='\t', index=True)

# SLU2 preanalysis
## Import data
### > 50 lines in .gff

In [ ]:
path_to_images = '/scratch/project_2006608/Methylation/notebooks/UMAP_WW_above50/'

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU2_matrices_top50/flattened/SLU2_concat_matrices_top50.tsv'
SLU2_matrices = pd.read_csv(file_path, sep='\t', index_col=0, low_memory=False)

In [ ]:
print(SLU2_matrices.shape[0])

In [ ]:
print(SLU2_matrices.shape)
SLU2_matrices.head()

In [ ]:
SLU2_df = SLU2_matrices.loc[(SLU2_matrices.iloc[:, :492] != 0).any(axis=1)]
print(SLU2_df.shape)

In [ ]:
SLU2_df['sample'].value_counts()
print(SLU2_df.iloc[:, :-1])

## Attach metadata
### Mod counts

In [ ]:
# Bring the mod counts from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU2_contigs/SLU2_mod_counts.txt'

SLU2_df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU2_df_mod_counts.shape[0])
SLU2_df_mod_counts.head()

In [ ]:
# Append to merged_data.tsv
SLU2_df_ext = SLU2_df.copy()
SLU2_df_ext.head()
SLU2_df_mod_counts.head()

# Reorder to match
SLU2_df_ordered = SLU2_df_mod_counts.loc[SLU2_df_ext.index]

# Check the min mod counts
SLU2_df_ordered['mod_count'].min()

In [ ]:
# Log transform
SLU2_df_ordered['mod_count_log'] = np.log2(SLU2_df_ordered['mod_count'])

SLU2_df_mod_counts = pd.concat([SLU2_df_ext, SLU2_df_ordered], axis=1)
print(SLU2_df_mod_counts.iloc[:, :-3])
SLU2_df_mod_counts.head()
SLU2_df = SLU2_df_mod_counts.copy()

### ARG counts (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU2_contigs/SLU2_ARG_counts.txt'

SLU2_df_ARG_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU2_df_ARG_counts.shape[0])
SLU2_df_ARG_counts.head()

In [ ]:
## Append to merged_data.tsv
SLU2_df_ext = SLU2_df.copy()
SLU2_df_ext.head()
SLU2_df_ARG_counts.head()

# Reorder to match
SLU2_df_ordered = SLU2_df_ARG_counts.loc[SLU2_df_ext.index]

SLU2_df_ARG_counts = pd.concat([SLU2_df_ext, SLU2_df_ordered], axis=1)
print(SLU2_df_ARG_counts.iloc[:, :-4])
SLU2_df_ARG_counts.head()
SLU2_df = SLU2_df_ARG_counts.copy()

### ARG names (ResFinder)

In [ ]:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU2_contigs/SLU2_ARG_names.txt'

SLU2_df_ARG_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU2_df_ARG_names.shape[0])
SLU2_df_ARG_names.head()

# Replace NaN with empty
SLU2_df_ARG_names = SLU2_df_ARG_names.fillna('')

In [ ]:
## Append to merged_data.tsv
SLU2_df_ext = SLU2_df.copy()
SLU2_df_ext.head()
SLU2_df_ARG_names.head()

# Reorder to match
SLU2_df_ordered = SLU2_df_ARG_names.loc[SLU2_df_ext.index]

SLU2_df_ARG_names = pd.concat([SLU2_df_ext, SLU2_df_ordered], axis=1)
print(SLU2_df_ARG_names.iloc[:, :-5])
SLU2_df_ARG_names.head()
SLU2_df = SLU2_df_ARG_names.copy()

### Contig lengths

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU2_contigs/SLU2_contigs_lengths.txt'

SLU2_df_contigs_lengths = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU2_df_contigs_lengths.shape[0])
SLU2_df_contigs_lengths.head()

In [ ]:
## Append to merged_data.tsv
SLU2_df_ext = SLU2_df.copy()
SLU2_df_ext.head()
SLU2_df_contigs_lengths.head()

# Reorder to match
SLU2_df_ordered = SLU2_df_contigs_lengths.loc[SLU2_df_ext.index]

# Log transform
SLU2_df_ordered['length_sqrt'] = np.sqrt(SLU2_df_ordered['length'])

# Concat
SLU2_df_contigs_lengths = pd.concat([SLU2_df_ext, SLU2_df_ordered], axis=1)
print(SLU2_df_contigs_lengths.iloc[:, :-7])
SLU2_df_contigs_lengths.head()
SLU2_df = SLU2_df_contigs_lengths.copy()

### fARGene results

In [ ]:
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/WW_data/SLU2_contigs/SLU2_fARGene_names.txt'

SLU2_df_fARGene_names = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(SLU2_df_fARGene_names.shape[0])
SLU2_df_fARGene_names.head()

# Replace NaN with empty
SLU2_df_fARGene_names = SLU2_df_fARGene_names.fillna('')
SLU2_df_fARGene_names.head()

In [ ]:
## Append to merged_data.tsv
SLU2_df_ext = SLU2_df.copy()
SLU2_df_ext.head()
SLU2_df_fARGene_names.head()

# Reorder to match
SLU2_df_ordered = SLU2_df_fARGene_names.loc[SLU2_df_ext.index]

SLU2_df_fARGene_names = pd.concat([SLU2_df_ext, SLU2_df_ordered], axis=1)
print(SLU2_df_fARGene_names.iloc[:, :-8])
SLU2_df_fARGene_names.head()
SLU2_df = SLU2_df_fARGene_names.copy()

### Explore ARGs

In [ ]:
# Print those with erm(F)_3
erm_F = SLU2_df[SLU2_df['ARG_name'].str.contains('erm(F)_3', case=False, na=False, regex=False)]
print(erm_F)

In [ ]:
SLU2_df.head()
#print(SLU1_df.iloc[:, :-8])
#print(SLU1_df.index)

## Draw UMAP

In [ ]:
n_neighbors = [20]
min_dist = [0.1]
#colors = [0, 1, 2, 3]
#color_map = {0: '#8ce6e9', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
#custom_colors = [color_map[val] for val in colors]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(SLU2_df.iloc[:, :-8])
        SLU2_UMAP_df  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': SLU2_df.index,
            'mod_count':SLU2_df['mod_count'],
            'mod_count_log':SLU2_df['mod_count_log'],
            'ARG_name':SLU2_df['ARG_name'],
            'ARG_count':SLU2_df['ARG_count'],
            'contig_length':SLU2_df['length'],
            'contig_length_sqrt':SLU2_df['length_sqrt'],
            'fARGene_class':SLU2_df['fARGene']
        })

        SLU2_UMAP_df['ARG_count'] = SLU2_UMAP_df['ARG_count'].astype(str)
        
        fig = px.scatter(SLU2_UMAP_df, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='mod_count_log',
                            #color='ARG_count',
                            title=f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_continuous_scale=px.colors.sequential.Rainbow,
                            #color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()
        fig.write_image(f'UMAP_WW_above50/SLU2_UMAP_{n}_{m}_mod_counts_lengths.png')
        fig.write_html(f'UMAP_WW_above50/SLU2_UMAP_{n}_{m}_mod_counts_lengths.html')

## Exclude 'möykky'

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_UMAP_df_focused1 = SLU2_UMAP_df.loc[(SLU2_UMAP_df['UMAP1']>= -5) & (SLU2_UMAP_df['UMAP1']<= 22)
    & (SLU2_UMAP_df['UMAP2']>= 12.9) & (SLU2_UMAP_df['UMAP2']<= 22)]

SLU2_UMAP_df_focused2 = SLU2_UMAP_df.loc[(SLU2_UMAP_df['UMAP1']>= -5) & (SLU2_UMAP_df['UMAP1']<= 13)
    & (SLU2_UMAP_df['UMAP2']>= 5) & (SLU2_UMAP_df['UMAP2']<= 12.9)]

SLU2_UMAP_df_focused3 = SLU2_UMAP_df.loc[(SLU2_UMAP_df['UMAP1']>= -5) & (SLU2_UMAP_df['UMAP1']<= 22)
    & (SLU2_UMAP_df['UMAP2']>= -7) & (SLU2_UMAP_df['UMAP2']<= 5)]

SLU2_UMAP_df_focused = pd.concat([SLU2_UMAP_df_focused1, SLU2_UMAP_df_focused2, SLU2_UMAP_df_focused3], ignore_index=False)

# Check
SLU2_UMAP_df_focused.head()

SLU2_df_focused = SLU2_df[SLU2_df.index.isin(SLU2_UMAP_df_focused['contig'])]
print(SLU2_df_focused)

In [ ]:
# Save contig IDs
SLU2_focused_contigs = SLU2_df_focused.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_focused_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_focused_contigs:
        file.write(f"{item}\n")

In [ ]:
print(SLU2_df_focused.iloc[:, :-8])

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#f5c7bf', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]
category_order = ["0", "1", "2", "3"]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(SLU2_df_focused.iloc[:, :-8])
        SLU2_UMAP_df_focused  = pd.DataFrame({
            'UMAP1': embedding[:, 0],
            'UMAP2': embedding[:, 1],
            'contig': SLU2_df_focused.index,
            'mod_count':SLU2_df_focused['mod_count'],
            'mod_count_log':SLU2_df_focused['mod_count_log'],
            'ARG_name':SLU2_df_focused['ARG_name'],
            'ARG_count':SLU2_df_focused['ARG_count'],
            'contig_length':SLU2_df_focused['length'],
            'contig_length_sqrt':SLU2_df_focused['length_sqrt'],
            'fARGene_class':SLU2_df_focused['fARGene']
        })

        SLU2_UMAP_df_focused['ARG_count'] = SLU2_UMAP_df_focused['ARG_count'].astype(str)

        fig = px.scatter(SLU2_UMAP_df_focused, 
                            x='UMAP1', 
                            y='UMAP2', 
                            color='ARG_count',
                            category_orders={'ARG_count': category_order},
                            title=f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}', 
                            color_discrete_sequence=custom_colors,
                            hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                                       'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                        size='contig_length_sqrt')
        title = f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}'
        fig.update_layout(
            height=1700,
            width=1200,
            title_text=title,
            showlegend=True,
            legend=dict(
                x=0.5,
                y=-0.05,
                traceorder="normal",
                xanchor='center',
                yanchor='top',
                orientation='h'
            ),
            template='simple_white',
            xaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,  
                linecolor='black', 
                linewidth=1,
                mirror=True
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='lightgray',
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True
            )
        )
        fig.show()

## Save data

In [ ]:
# Check
#SLU2_df_focused.head()
#SLU2_UMAP_df_focused.head()

In [ ]:
# Save data
SLU2_df_focused.to_csv('UMAP_WW_above50/SLU2_df_focused.csv', sep='\t', index=True)
SLU2_UMAP_df_focused.to_csv('UMAP_WW_above50/SLU2_UMAP_df_focused.csv', sep='\t', index=True)

# SLU2: Read in and plot

In [ ]:
# Read data
SLU2_df_focused = pd.read_csv('UMAP_WW_above50/SLU2_df_focused.csv', sep='\t', index_col=0, low_memory=False)
SLU2_UMAP_df_focused = pd.read_csv('UMAP_WW_above50/SLU2_UMAP_df_focused.csv', sep='\t', index_col=0, low_memory=False)

# Check
SLU2_df_focused.head()
SLU2_UMAP_df_focused.head()

## Plot ARG counts

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = [0, 1, 2, 3]
color_map = {0: '#f5c7bf', 1: '#fa7a31', 2: '#eb340f', 3: '#d01af5'}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

SLU2_UMAP_df_focused['ARG_count'] = SLU2_UMAP_df_focused['ARG_count'].astype(str)
category_order = ["0", "1", "2", "3"]

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(SLU2_UMAP_df_focused.iloc[:, :-8])
        
fig = px.scatter(SLU2_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='ARG_count',
                 title=f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}',
                 category_orders={'ARG_count': category_order},
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/SLU2_UMAP_{n}_{m}_focused_ARG_counts.png')
#fig.write_html(f'UMAP_WW_above50/SLU2_UMAP_{n}_{m}_focused_ARG_counts.html')

### Extract clusters
#### C1

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C1 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 7.2) & (SLU2_UMAP_df_focused['UMAP1']<= 7.25)
    & (SLU2_UMAP_df_focused['UMAP2']>= -6.95) & (SLU2_UMAP_df_focused['UMAP2']<= -6.9)]

# Check
SLU2_df_focused_C1.head()

SLU2_df_C1 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C1['contig'])]
print(SLU2_df_C1)

In [ ]:
# Save contig IDs
SLU2_C1_contigs = SLU2_df_C1.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C1_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C1_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C2

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C2 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 7.33) & (SLU2_UMAP_df_focused['UMAP1']<= 7.36)
    & (SLU2_UMAP_df_focused['UMAP2']>= -4.47) & (SLU2_UMAP_df_focused['UMAP2']<= -4.45)]

# Check
SLU2_df_focused_C2.head()

SLU2_df_C2 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C2['contig'])]
print(SLU2_df_C2)

In [ ]:
# Save contig IDs
SLU2_C2_contigs = SLU2_df_C2.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C2_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C2_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C3

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C3 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 4.1) & (SLU2_UMAP_df_focused['UMAP1']<= 4.2)
    & (SLU2_UMAP_df_focused['UMAP2']>= -7.8) & (SLU2_UMAP_df_focused['UMAP2']<= -7.76)]

# Check
SLU2_df_focused_C3.head()

SLU2_df_C3 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C3['contig'])]
print(SLU2_df_C3)

In [ ]:
# Save contig IDs
SLU2_C3_contigs = SLU2_df_C3.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C3_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C3_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C4

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C4 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 0.17) & (SLU2_UMAP_df_focused['UMAP1']<= 0.18)
    & (SLU2_UMAP_df_focused['UMAP2']>= 12.65) & (SLU2_UMAP_df_focused['UMAP2']<= 12.7)]

# Check
SLU2_df_focused_C4.head()

SLU2_df_C4 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C4['contig'])]
print(SLU2_df_C4)

In [ ]:
# Save contig IDs
SLU2_C4_contigs = SLU2_df_C4.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C4_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C4_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C5

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C5 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= -0.85) & (SLU2_UMAP_df_focused['UMAP1']<= -0.8)
    & (SLU2_UMAP_df_focused['UMAP2']>= 4.58) & (SLU2_UMAP_df_focused['UMAP2']<= 4.6)]

# Check
SLU2_df_focused_C5.head()

SLU2_df_C5 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C5['contig'])]
print(SLU2_df_C5)

In [ ]:
# Save contig IDs
SLU2_C5_contigs = SLU2_df_C5.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C5_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C5_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C6

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C6 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 14.565) & (SLU2_UMAP_df_focused['UMAP1']<= 14.571)
    & (SLU2_UMAP_df_focused['UMAP2']>= 3.128) & (SLU2_UMAP_df_focused['UMAP2']<= 3.135)]

# Check
SLU2_df_focused_C6.head()

SLU2_df_C6 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C6['contig'])]
print(SLU2_df_C6)

In [ ]:
# Save contig IDs
SLU2_C6_contigs = SLU2_df_C6.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C6_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C6_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C7

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C7 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= -6.6) & (SLU2_UMAP_df_focused['UMAP1']<= -6.4)
    & (SLU2_UMAP_df_focused['UMAP2']>= 8.44) & (SLU2_UMAP_df_focused['UMAP2']<= 8.55)]

# Check
SLU2_df_focused_C7.head()

SLU2_df_C7 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C7['contig'])]
print(SLU2_df_C7)

In [ ]:
# Save contig IDs
SLU2_C7_contigs = SLU2_df_C7.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C7_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C7_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C8

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C8 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= -0.22) & (SLU2_UMAP_df_focused['UMAP1']<= -0.15)
    & (SLU2_UMAP_df_focused['UMAP2']>= 0.679) & (SLU2_UMAP_df_focused['UMAP2']<= 0.74)]

# Check
SLU2_df_focused_C8.head()

SLU2_df_C8 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C8['contig'])]
print(SLU2_df_C8)

In [ ]:
# Save contig IDs
SLU2_C8_contigs = SLU2_df_C8.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C8_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C8_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C9

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C9 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= -0.02) & (SLU2_UMAP_df_focused['UMAP1']<= 0)
    & (SLU2_UMAP_df_focused['UMAP2']>= 0.845) & (SLU2_UMAP_df_focused['UMAP2']<= 0.855)]

# Check
SLU2_df_focused_C9.head()

SLU2_df_C9 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C9['contig'])]
print(SLU2_df_C9)

In [ ]:
# Save contig IDs
SLU2_C9_contigs = SLU2_df_C9.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C9_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C9_contigs:
        file.write(f"{item}\n")

### Extract clusters
#### C10

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C10 = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 9.05) & (SLU2_UMAP_df_focused['UMAP1']<= 9.1)
    & (SLU2_UMAP_df_focused['UMAP2']>= 5.579) & (SLU2_UMAP_df_focused['UMAP2']<= 5.595)]

# Check
SLU2_df_focused_C10.head()

SLU2_df_C10 = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C10['contig'])]
print(SLU2_df_C10)

In [ ]:
# Save contig IDs
SLU2_C10_contigs = SLU2_df_C10.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C10_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C10_contigs:
        file.write(f"{item}\n")

## Plot fARGene

In [ ]:
n_neighbors = [20]
min_dist = [0.1]

colors = ['', 'class_a', 'class_b_1_2', 'class_c', 'class_d_1', 'class_d_2', 'mph', 'qnr', 'tet_efflux',
          'tet_rpg', 'tet_enzyme', 'erm_type_a', 'erm_type_f', 'aminoglycoside_model_a', 'aminoglycoside_model_b',
          'aminoglycoside_model_c', 'aminoglycoside_model_d', 'aminoglycoside_model_e', 'aminoglycoside_model_f',
          'aminoglycoside_model_g', 'aminoglycoside_model_h', 'aminoglycoside_model_i']

color_map = {'': '#f5c7bf', 'class_a': "#fabefa", 'class_b_1_2': "#d86950", 'class_c': "#ef360c", 'class_d_1': "#a27faf", 'class_d_2': "#c308a4", 'mph': "#e8db16",
             'qnr': "black", 'tet_efflux': "#04c60a", 'tet_rpg': "#1e7e21", 'tet_enzyme': "#779e78", 'erm_type_a': "#66e4e4", 'erm_type_f': "#25a5a5",
             'aminoglycoside_model_a': "#e8db16", 'aminoglycoside_model_b': "#1656e8", 'aminoglycoside_model_c': "#0f378e",
             'aminoglycoside_model_d': "#86a4eb", 'aminoglycoside_model_e': "#5b48d8", 'aminoglycoside_model_f': "#146eb4",
             'aminoglycoside_model_g': "#6f87f3", 'aminoglycoside_model_h': "#85baec", 'aminoglycoside_model_i': "#04bdfe"}

# Map colors to each data point
custom_colors = [color_map[val] for val in colors]

SLU2_UMAP_df_focused['fARGene_class'] = SLU2_UMAP_df_focused['fARGene_class'].astype(str)
SLU2_UMAP_df_focused['fARGene_class'] = SLU2_UMAP_df_focused['fARGene_class'].replace('nan', '')

for n in n_neighbors:
    for m in min_dist:
        reducer = umap.UMAP(n_neighbors=n, min_dist=m, random_state=seed)
        embedding = reducer.fit_transform(SLU2_UMAP_df_focused.iloc[:, :-8])

fig = px.scatter(SLU2_UMAP_df_focused, 
                 x='UMAP1', 
                 y='UMAP2', 
                 color='fARGene_class',
                 title=f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}', 
                 color_discrete_sequence=custom_colors,
                 hover_data={'contig': True, 'mod_count': True, 'ARG_name': True,
                             'ARG_count': True, 'contig_length': True, 'fARGene_class': True},
                size='contig_length_sqrt')
title = f' Wastewater SLU2 - UMAP with n_neighbors={n}, min_dist={m}'
fig.update_layout(height=1700, width=1200, title_text=title, showlegend=True, 
                  legend=dict(x=0.5, y=-0.05, traceorder="normal",xanchor='center', yanchor='top', orientation='h'), 
                  template='simple_white', 
                  xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True),
                  yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showline=True, linecolor='black', linewidth=1, mirror=True))
fig.show()
#fig.write_image(f'UMAP_WW_above50/SLU2_UMAP_{n}_{m}_fARGene.png')
#fig.write_html(f'UMAP_WW_above50/SLU2_UMAP_{n}_{m}_fARGene.html')

### Extract clusters 
#### C1f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C1f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 4.5) & (SLU2_UMAP_df_focused['UMAP1']<= 5)
    & (SLU2_UMAP_df_focused['UMAP2']>= 4.8) & (SLU2_UMAP_df_focused['UMAP2']<= 5)]

# Check
SLU2_df_focused_C1f.head()

SLU2_df_C1f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C1f['contig'])]
print(SLU2_df_C1f)

In [ ]:
# Save contig IDs
SLU2_C1f_contigs = SLU2_df_C1f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C1f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C1f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C2f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C2f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 5.8) & (SLU2_UMAP_df_focused['UMAP1']<= 6.2)
    & (SLU2_UMAP_df_focused['UMAP2']>= 0.2) & (SLU2_UMAP_df_focused['UMAP2']<= 0.4)]

# Check
SLU2_df_focused_C2f.head()

SLU2_df_C2f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C2f['contig'])]
print(SLU2_df_C2f)

In [ ]:
# Save contig IDs
SLU2_C2f_contigs = SLU2_df_C2f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C2f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C2f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C3f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C3f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 4.34) & (SLU2_UMAP_df_focused['UMAP1']<= 4.35)
    & (SLU2_UMAP_df_focused['UMAP2']>= -7.33) & (SLU2_UMAP_df_focused['UMAP2']<= -7.31)]

# Check
SLU2_df_focused_C3f.head()

SLU2_df_C3f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C3f['contig'])]
print(SLU2_df_C3f)

In [ ]:
# Save contig IDs
SLU2_C3f_contigs = SLU2_df_C3f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C3f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C3f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C4f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C4f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= -5.75) & (SLU2_UMAP_df_focused['UMAP1']<= -5.65)
    & (SLU2_UMAP_df_focused['UMAP2']>= -9.82) & (SLU2_UMAP_df_focused['UMAP2']<= -9.76)]

# Check
SLU2_df_focused_C4f.head()

SLU2_df_C4f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C4f['contig'])]
print(SLU2_df_C4f)

In [ ]:
# Save contig IDs
SLU2_C4f_contigs = SLU2_df_C4f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C4f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C4f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C5f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C5f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 6.28) & (SLU2_UMAP_df_focused['UMAP1']<= 6.32)
    & (SLU2_UMAP_df_focused['UMAP2']>= 7.37) & (SLU2_UMAP_df_focused['UMAP2']<= 7.38)]

# Check
SLU2_df_focused_C5f.head()

SLU2_df_C5f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C5f['contig'])]
print(SLU2_df_C5f)

In [ ]:
# Save contig IDs
SLU2_C5f_contigs = SLU2_df_C5f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C5f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C5f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C6f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C6f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= 6.6) & (SLU2_UMAP_df_focused['UMAP1']<= 6.7)
    & (SLU2_UMAP_df_focused['UMAP2']>= 7.0) & (SLU2_UMAP_df_focused['UMAP2']<= 7.2)]

# Check
SLU2_df_focused_C6f.head()

SLU2_df_C6f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C6f['contig'])]
print(SLU2_df_C6f)

In [ ]:
# Save contig IDs
SLU2_C6f_contigs = SLU2_df_C6f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C6f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C6f_contigs:
        file.write(f"{item}\n")

### Extract clusters 
#### C7f (fARGene)

In [ ]:
# Extract based on UMAP1 (x axis) & UMAP2 (y axis) values:
SLU2_df_focused_C7f = SLU2_UMAP_df_focused.loc[(SLU2_UMAP_df_focused['UMAP1']>= -0.846) & (SLU2_UMAP_df_focused['UMAP1']<= -0.836)
    & (SLU2_UMAP_df_focused['UMAP2']>= 4.617) & (SLU2_UMAP_df_focused['UMAP2']<= 4.6185)]

# Check
SLU2_df_focused_C7f.head()

SLU2_df_C7f = SLU2_df_focused[SLU2_df_focused.index.isin(SLU2_df_focused_C7f['contig'])]
print(SLU2_df_C7f)

In [ ]:
# Save contig IDs
SLU2_C7f_contigs = SLU2_df_C7f.index.to_list()

directory = 'UMAP_WW_above50'
file_path = os.path.join(directory, 'SLU2_C7f_contigs.txt')

with open(file_path, 'w') as file:
    for item in SLU2_C7f_contigs:
        file.write(f"{item}\n")